## Run this notebook

You can launch this notbook using mybinder, by clicking the button below.

<a href="https://mybinder.org/v2/gh/NASA-IMPACT/veda-docs/HEAD?labpath=example-notebooks%2Fwfs.ipynb">
<img src="https://mybinder.org/badge_logo.svg" alt="Binder" title="A cute binder" width="150"/> 
</a>

## Approach

   1. Use `OWSLib` to determine what data is available and inspect the metadata
   2. Use `OWSLib` to filter and read the data
   3. Use `geopandas` and `folium` to analyze and plot the data

Note that the default examples environment is missing one requirement: `oswlib`. We can `pip install` that before we move on. 

In [1]:
!pip install OWSLib==0.28.1

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
from owslib.ogcapi.features import Features
import geopandas as gpd
import datetime as dt
from datetime import datetime, timedelta

## Look at the data that is availible through the OGC API

The datasets that are distributed throught the OGC API are organized into collections.
We can display the collections with the command: 

In [3]:
OGC_URL = "https://firenrt.delta-backend.com"

w = Features(url=OGC_URL)
w.feature_collections()

['public.eis_fire_newfirepix',
 'public.eis_fire_perimeter',
 'public.eis_fire_fireline',
 'public.st_subdivide',
 'public.st_hexagongrid',
 'public.st_squaregrid']

We will focus on the `public.eis_fire_fireline` collection and the `public.eis_fire_perimeter` collection. 

### Inspect the metatdata for public.eis_fire_perimeter collection

We can access information that drescribes the `public.eis_fire_perimeter`. 

In [4]:
perm = w.collection("public.eis_fire_perimeter")

We are particularly interested in the spatial and temporal extents of the data.

In [5]:
perm["extent"]

{'spatial': {'bbox': [[-125.16691790604779,
    24.116061807989844,
    -61.8556830871285,
    49.37712098567962]],
  'crs': 'http://www.opengis.net/def/crs/OGC/1.3/CRS84'},
 'temporal': {'interval': [['2023-03-30T00:00:00+00:00',
    '2023-04-19T00:00:00+00:00']],
  'trs': 'http://www.opengis.net/def/uom/ISO-8601/0/Gregorian'}}

In addition to getting metadata about the data we can access the queryable fields. Each of these fields will represent a column in our dataframe.

In [6]:
perm_q = w.collection_queryables("public.eis_fire_perimeter")
perm_q["properties"]

{'wkb_geometry': {'$ref': 'https://geojson.org/schema/Geometry.json'},
 'ogc_fid': {'name': 'ogc_fid', 'type': 'number'},
 'n_pixels': {'name': 'n_pixels', 'type': 'number'},
 'n_newpixels': {'name': 'n_newpixels', 'type': 'number'},
 'farea': {'name': 'farea', 'type': 'number'},
 'fperim': {'name': 'fperim', 'type': 'number'},
 'flinelen': {'name': 'flinelen', 'type': 'number'},
 'duration': {'name': 'duration', 'type': 'number'},
 'pixden': {'name': 'pixden', 'type': 'number'},
 'meanfrp': {'name': 'meanfrp', 'type': 'number'},
 'isactive': {'name': 'isactive', 'type': 'number'},
 't': {'name': 't', 'type': 'string'},
 'fireid': {'name': 'fireid', 'type': 'number'}}

## Filter the data

It is always a good idea to do any data filtering as early as possible. In this example we know that we want the data for particular spatial and temporal extents. We can apply those and other filters using the `OWSLib` package.

In the below example we are:

 - choosing the `public.eis_fire_perimeter` collection 
 - subsetting it by space using the `bbox` parameter
 - subsetting it by time using the `datetime` parameter
 - filtering for fires over 5km^2 and over 2 days long using the `filter` parameter. The `filter` parameter lets us filter by the columns in 'public.eis_fire_perimeter' using SQL-style queries. 

NOTE: The `limit` parameter desginates the maximum number of objects the query will return. The default limit is 10, so if we want to all of the fire perimeters within certain conditions, we need to make sure that the limit is large.

In [7]:
## Get the most recent fire perimeters, and 7 days before most recent fire perimeter
most_recent_time = max(*perm["extent"]["temporal"]["interval"])
now = dt.datetime.strptime(most_recent_time, "%Y-%m-%dT%H:%M:%S+00:00")
last_week = now - dt.timedelta(weeks=1)
last_week = dt.datetime.strftime(last_week, "%Y-%m-%dT%H:%M:%S+00:00")
print("Most Recent Time =", most_recent_time)
print("Last week =", last_week)

Most Recent Time = 2023-04-19T00:00:00+00:00
Last week = 2023-04-12T00:00:00+00:00


In [8]:
perm_results = w.collection_items(
    "public.eis_fire_perimeter",  # name of the dataset we want
    bbox=["-106.8", "24.5", "-72.9", "37.3"],  # coodrinates of bounding box,
    datetime=[last_week + "/" + most_recent_time],  # date range
    limit=1000,  # max number of items returned
    filter="farea>5 AND duration>2",  # additional filters based on queryable fields
)

The result is a dictionary containing all of the data and some summary fields. We can look at the keys to see what all is in there.

In [9]:
perm_results.keys()

dict_keys(['type', 'id', 'title', 'description', 'numberMatched', 'numberReturned', 'links', 'features'])

For instance you can check the total number of matched items and make sure that it is equal to the number of returned items. This is how you know that the `limit` you defined above is high enough.

In [10]:
perm_results["numberMatched"] == perm_results["numberReturned"]

True

You can also access the data directly in the browser or in an HTTP GET call using the constructed link.

In [11]:
perm_results["links"][1]["href"]

'https://firenrt.delta-backend.com/collections/public.eis_fire_perimeter/items?bbox=-106.8%2C24.5%2C-72.9%2C37.3&datetime=2023-04-12T00%3A00%3A00%2B00%3A00%2F2023-04-19T00%3A00%3A00%2B00%3A00&limit=1000&filter=farea%3E5+AND+duration%3E2'

## Read data

In addition to all the summary fields, the `perm_results` dict contains all the data. We can pass the data into geopandas to make it easier to interact with.

In [12]:
df = gpd.GeoDataFrame.from_features(perm_results["features"])
df

,geometry,ogc_fid,n_pixels,n_newpixels,farea,fperim,flinelen,duration,pixden,meanfrp,isactive,t,fireid
0,"POLYGON ((-94.92952 34.21025, -94.92951 34.210...",1627,175,0,36.914361,40.545199,0.0,3.0,4.740702,0.0,1,2023-04-14T12:00:00,47998
1,"MULTIPOLYGON (((-94.21166 34.82102, -94.21169 ...",1670,187,0,30.058442,49.606250,0.0,4.0,6.221214,0.0,1,2023-04-14T12:00:00,46850
2,"POLYGON ((-93.51528 35.85425, -93.51521 35.854...",2091,66,0,13.140147,19.700381,0.0,3.0,5.022775,0.0,1,2023-04-14T12:00:00,47822
3,"POLYGON ((-94.68205 35.07044, -94.68205 35.070...",2131,36,0,6.523911,10.634670,0.0,6.0,5.518162,0.0,1,2023-04-17T00:00:00,47508
4,"POLYGON ((-91.46354 37.08346, -91.46334 37.083...",2934,89,0,8.150479,11.893227,0.0,4.0,10.919603,0.0,0,2023-04-12T12:00:00,45190
5,"POLYGON ((-81.35660 32.81974, -81.35677 32.819...",4236,72,0,17.501778,26.349821,0.0,34.0,4.113868,0.0,1,2023-04-18T12:00:00,31357
6,"POLYGON ((-81.06540 32.37262, -81.06541 32.372...",4313,33,0,5.214923,10.994579,0.0,5.0,6.327994,0.0,1,2023-04-17T12:00:00,48921
7,"MULTIPOLYGON (((-96.18329 36.97956, -96.18309 ...",9873,347,0,51.520094,56.936577,0.0,20.0,6.735236,0.0,1,2023-04-17T12:00:00,38904
8,"MULTIPOLYGON (((-96.44047 36.84058, -96.44047 ...",9976,109,0,19.687421,22.361965,0.0,7.0,5.536530,0.0,0,2023-04-13T12:00:00,43579
9,"POLYGON ((-96.24797 36.88458, -96.24777 36.884...",9977,160,0,14.874136,16.226787,0.0,6.0,10.756927,0.0,1,2023-04-17T12:00:00,47819


## Explore data

We can quickly explore the data by setting the coordinate reference system (`crs`) and using `.explore()`

In [ ]:
df = df.set_crs("EPSG:4326")
df.explore()

## Visualize Most Recent Fire Perimeters with Firelines

If we wanted to combine collections to make more informative analyses, we can use some of the same principles. 

First we'll get the queryable fields, and the extents:

In [ ]:
fline_q = w.collection_queryables("public.eis_fire_fireline")
fline_collection = w.collection("public.eis_fire_fireline")
fline_q["properties"]

### Read
Then we'll use those fields to get most recent fire perimeters and fire lines.

In [ ]:
perm_results = w.collection_items(
    "public.eis_fire_perimeter",
    datetime=most_recent_time,
    limit=1000,
)
perimeters = gpd.GeoDataFrame.from_features(perm_results["features"])

## Get the most recent fire lines
perimeter_ids = perimeters.fireid.unique()
perimeter_ids = ",".join(map(str, perimeter_ids))

fline_results = w.collection_items(
    "public.eis_fire_fireline",
    limit=1000,
    filter="fireid IN ("
    + perimeter_ids
    + ")",  # only the fires from the fire perimeter query above
)
fline = gpd.GeoDataFrame.from_features(fline_results["features"])

### Visualize

In [ ]:
perimeters = perimeters.set_crs("epsg:4326")
fline = fline.set_crs("epsg:4326")

m = perimeters.explore()
m = fline.explore(m=m, color="orange")
m

## Download Data

Downloading pre-filtered data may be useful for working locally, or for working with the data in GIS software. 


We can download the dataframe we made by writing it out into a shapefile or into a GeoJSON file.  

    perimeters.to_file('perimeters.shp') 
    perimeters.to_file('perimeters.geojson', driver='GeoJSON')

## About the Data


The fire data shown is generated by the FEDs algorithm. The FEDs algorithm tracks fire movement and severity by ingesting observations from the VIIRS thermal sensors on the Suomi NPP and NOAA-20 satellites. This algorithm uses raw VIIRS observations to generate a polygon of the fire, locations of the active fire line, and estimates of fire mean Fire Radiative Power (FRP). The VIIRS sensors overpass at ~1:30 AM and PM local time, and provide estimates of fire evolution ~ every 12 hours. The data produced by this algorithm describe where fires are in space and how fires evolve through time. This CONUS-wide implementation of the FEDs algorithm is based on [Chen et al 2020’s](https://www.nature.com/articles/s41597-022-01343-0) algorithm for California.


The data produced by this algorithm is considered experimental. 


## Collection Information

**public.eis_fire_perimeter**

Perimeter of cumulative fire-area.

| Column      | Description | Unit      |
| ----------- | ----------- | ----------- |
| meanfrp      | Mean fire radiative power. The weighted sum of the fire radiative power detected at each new pixel, divided by the number of pixels.   If no new pixels are detected, meanfrp is set to zero. | MW/(pixel area)  |
| t   | Time of most recent VIIRS detection, corrected to noon and midnight. API delivers all perimeters with a VIIRS detection in the last 20 days.    | Datetime. yyyy-mm-ddThh:mm:ss. Local time.  |
| fireid   | Fire ID. Unique for each fire. Matches fireid. |Numeric ID |
| pixden   |Number of pixels divided by area of perimeter.   | pixels/Km^2|
| duration  |Number of days since first observation of fire. Fires with a single observation have a duration of zero. |Days|
| flinelen   | Length of active fire line, based on new pixels. If no new pixels are detected, flinelen is set to zero.   |Km |
| fperim   | Length of fire perimeter.        | Km   |
| farea   | Area within fire perimeter.        |Km^2    |
| n_newpixels   | Number of pixels newly detected since last overpass.      |pixels  |
| n_pixels   | Number of pixel-detections in history of fire. | pixels   |
| isactive   | Have new fire pixels been detected in the last 5 days?     | Boolean |
| ogc_fid   |The ID used by the OGC API to sort perimeters.       | Numeric ID   |
| geometry  |The shape of the perimeter.    | Geometry    |


**public.eis_fire_fireline**

Active fire line as estimated by new VIIRS detections.

**public.eis_fire_newfirepix**

New pixel detections that inform the most recent time-step’s perimeter and fireline calculation.

| Column      | Description | Unit      |
| ----------- | ----------- | ----------- |
| fireid   | ID of fire pixel associated with.      | Numeric ID |
| t   | Time of most recent VIIRS detection, corrected to noon and midnight. API delivers all perimeters with a VIIRS detection in the last 20 days.| Datetime. yyyy-mm-ddThh:mm:ss. Local time.|
| mergeid  | ID used to connect pixels to perimeters. Matches fireid      | Numeric ID      |
| ogc_fid   |The ID used by the OGC API to sort pixels.      | Numeric ID     |
